In [6]:
# import sqlite3


In [3]:
# with sqlite3.connect("data/papers.db") as conn:
#     c = conn.cursor()
#     c.execute("Select * from papers")
    
#     # Fetch all results
#     all_rows = c.fetchall()
#     # print(all_rows)
    
#     # Check if any rows were fetched
#     if all_rows:
#         print("Fetched records:")
#         for row in all_rows:
#             print(row)
#     else:
#         print("No records found.")

In [4]:
# for paper in all_rows:
#     print("\n" + paper[1])
#     print(paper[-1])


In [125]:
from langchain_community.document_loaders import PyPDFLoader
from get_data import get_paper_list

# links = get_paper_list()
links = [paper[2] for paper in all_rows]
titles = [paper[1] for paper in all_rows]



In [126]:
links[0], titles[0]

('https://arxiv.org/pdf/2402.13616v2.pdf',
 'YOLOv9: Learning What You Want to Learn Using Programmable Gradient Information')

In [188]:
docs = []
for link, title in zip(links,titles):
    loader = PyPDFLoader(link)
    doc = loader.load_and_split()
    for i in range(len(doc)):
        doc[i].metadata['title'] = title
    docs.extend(doc)

In [189]:
docs[26].metadata

{'source': 'https://arxiv.org/pdf/2402.15151v1.pdf',
 'page': 0,
 'title': 'Where Visual Speech Meets Language: VSP-LLM Framework for Efficient and Context-Aware Visual Speech Processing'}

In [190]:
len(docs)

268

# RAG starts here (docs come in)

## Cost of embedding? 

In [24]:
import tiktoken

# Create encoder
encoder = tiktoken.get_encoding('cl100k_base')
tokens_per_docs = [len(encoder.encode(doc.page_content)) for doc in docs]


# Estimated cost = sum of tokens / 1000
cost_per_1000_tokens = 0.0001
cost = (sum(tokens_per_docs) / 1000) * cost_per_1000_tokens
cost


0.017815

## Try ArxivRetriever

In [1]:
from get_data import get_paper_info

meta = get_paper_info()
# meta[0]

for paper in meta:
    print("\n" + paper['title'])
    print(paper['authors'])
    print(paper['published'])
    print(paper['arxiv_link'])

    print("\n" + 'Summary:')
    print(paper['summary'])

## Vector database

In [38]:
# from langchain.retrievers import ParentDocumentRetriever
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [161]:
#Embed and store the texts
# Supplying a persist dicrectory will store the embeddings on disk
persist_direcory = 'data/vectordb'

embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=persist_direcory)


In [162]:
# Persist DB to disk
vectordb.persist()
vectordb = None

In [163]:
# Now can load and use the db
vectordb = Chroma(embedding_function=embeddings, persist_directory=persist_direcory)


## How to pull specific paper from vector database?

In [207]:
results = vectordb.similarity_search(query= "Where Visual Speech Meets Language: VSP-LLM Framework for Efficient and Context-Aware Visual Speech Processing")

for result in results:
    print(result.metadata)

{'page': 0, 'source': 'https://arxiv.org/pdf/2402.15151v1.pdf', 'title': 'Where Visual Speech Meets Language: VSP-LLM Framework for Efficient and Context-Aware Visual Speech Processing'}
{'page': 7, 'source': 'https://arxiv.org/pdf/2402.15151v1.pdf', 'title': 'Where Visual Speech Meets Language: VSP-LLM Framework for Efficient and Context-Aware Visual Speech Processing'}
{'page': 2, 'source': 'https://arxiv.org/pdf/2402.15151v1.pdf', 'title': 'Where Visual Speech Meets Language: VSP-LLM Framework for Efficient and Context-Aware Visual Speech Processing'}
{'page': 1, 'source': 'https://arxiv.org/pdf/2402.15151v1.pdf', 'title': 'Where Visual Speech Meets Language: VSP-LLM Framework for Efficient and Context-Aware Visual Speech Processing'}


## Make a retriever

In [210]:
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [209]:
retriever.search_type

'similarity'

In [211]:
docs = retriever.get_relevant_documents("Large Language Models")
len(docs)


10

In [216]:
docs

[Document(page_content='pages 358–373.\nJordan Hoffmann, Sebastian Borgeaud, Arthur Mensch,\nElena Buchatskaya, Trevor Cai, Eliza Rutherford,\nDiego de Las Casas, Lisa Anne Hendricks, Johannes\nWelbl, Aidan Clark, et al. 2022. Training compute-\noptimal large language models. ArXiv preprint .\nChristian Kohlschütter, Peter Fankhauser, and Wolfgang\nNejdl. 2010. Boilerplate detection using shallow text\nfeatures. In Proceedings of the Third International\nConference on Web Search and Web Data Mining,\nWSDM 2010, New York, NY, USA, February 4-6,\n2010 , pages 441–450.\nJunlong Li, Yiheng Xu, Lei Cui, and Furu Wei. 2021.\nMarkuplm: Pre-training of text and markup language\nfor visually-rich document understanding. ArXiv\npreprint .\nStephen Merity, Caiming Xiong, James Bradbury, and\nRichard Socher. 2017. Pointer sentinel mixture mod-\nels. In Proceedings of ICLR .', metadata={'page': 4, 'source': 'https://arxiv.org/pdf/2402.14652v1.pdf'}),
 Document(page_content='pages 358–373.\nJordan H

## Make a chain

In [234]:
from langchain.chains import RetrievalQAWithSourcesChain
# from langchain.chains import RetrievalQA

# from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
# from langchain.chains.combine_documents import create_stuff_documents_chain


# Create a chain to answer questions

qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm = ChatOpenAI(),
                                       chain_type='stuff',
                                       retriever=retriever,
                                       )

In [233]:
qa_chain.llm_chain.prompt_template

AttributeError: 'RetrievalQA' object has no attribute 'llm_chain'

In [222]:
## Cite sources

def process_llm_resoonse(llm_response):
    print(llm_response['answer'])
    print('\n\nSources:')
    # print(llm_response['sources'])
    for source in llm_response['sources'].split(','):
        print(source)


query = "What are the document titles?"
llm_response = qa_chain.invoke(query)
process_llm_resoonse(llm_response)



I don't know the answer.



Sources:



In [220]:
llm_response

{'question': 'What do these documents say about large language Models. Include sources and titles',
 'answer': 'The documents discuss various aspects of large language models, including their training, evaluation, applications, and potential. Titles include "Training compute-optimal large language models," "Prompting large language models with speech recognition abilities," "Assess and summarize: Improve outage understanding with large language models," "The rise and potential of large language model-based agents: A survey," and more.\n',
 'sources': 'https://arxiv.org/pdf/2402.14652v1.pdf, https://arxiv.org/pdf/2402.07939v3.pdf, https://arxiv.org/pdf/2402.03099v1.pdf'}

In [250]:
docs[0].metadata

{'Published': '2024-02-13',
 'Title': 'World Model on Million-Length Video And Language With RingAttention',
 'Authors': 'Hao Liu, Wilson Yan, Matei Zaharia, Pieter Abbeel',
 'Summary': 'Current language models fall short in understanding aspects of the world not\neasily described in words, and struggle with complex, long-form tasks. Video\nsequences offer valuable temporal information absent in language and static\nimages, making them attractive for joint modeling with language. Such models\ncould develop a understanding of both human textual knowledge and the physical\nworld, enabling broader AI capabilities for assisting humans. However, learning\nfrom millions of tokens of video and language sequences poses challenges due to\nmemory constraints, computational complexity, and limited datasets. To address\nthese challenges, we curate a large dataset of diverse videos and books,\nutilize the RingAttention technique to scalably train on long sequences, and\ngradually increase context s

## Parent document?

In [229]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter